# Data cleaning: HURDAT 2
This notebook takes the raw [HURDAT 2](https://www.aoml.noaa.gov/hrd/hurdat/Data_Storm.html) data and executes several cleaning operations. The result is a list of observations only for storms that reach hurricane or tropical storm status, and includes the observed storm's code, name, and date of formation in each row.

In [1]:
import numpy as np
import pandas as pd
import csv

In [2]:
def str2deg_lon(s):
	# NOAA longitude recorded as 0 to 360 deg east
	direction = s[-1]
	deg = float(s[:-1])
	return deg if direction == 'E' else 360 - deg

def str2deg_lat(s):
	# NOAA latitude recorded as -90 to 90 deg north
	direction = s[-1]
	deg = float(s[:-1])
	return deg if direction == 'N' else -deg

In [3]:
# read in CSV as a Python list
with open('../data/raw/hurdat2.csv', mode='r', newline='') as file:
    reader = csv.reader(file, delimiter=',')
    orig = list(reader)

# strip out whitespace from HTML
for i in range(len(orig)):
	for j in range(len(orig[i])):
		orig[i][j] = orig[i][j].strip()

orig.pop(0) # drop first empty row

# Combine section headers into observations, copy only observations to new list
lst = []
header = []
for row in orig:
	if len(row) == 4:
		header = row
	else:
		lst.append(header + row)

# Convert to DataFrame
df_cols = [
	"code",
	"name",
	"n_observations",
	"UNUSED",
	"date_string",
	"time_24h",
	"record_identifier",
	"status",
	"lat_string",
	"lon_string",
	"max_sus_wind_kt",
	"min_pressure_mb",
	"r_34kt_ne",
	"r_34kt_se",
	"r_34kt_sw",
	"r_34kt_nw",
	"r_50kt_ne",
	"r_50kt_se",
	"r_50kt_sw",
	"r_50kt_nw",
	"r_64kt_ne",
	"r_64kt_se",
	"r_64kt_sw",
	"r_64kt_nw",
	"r_max_sus"
]
df = pd.DataFrame(lst, columns=df_cols)

# convert datatypes
float_cols = [
	"max_sus_wind_kt",
	"min_pressure_mb",
	"r_34kt_ne",
	"r_34kt_se",
	"r_34kt_sw",
	"r_34kt_nw",
	"r_50kt_ne",
	"r_50kt_se",
	"r_50kt_sw",
	"r_50kt_nw",
	"r_64kt_ne",
	"r_64kt_se",
	"r_64kt_sw",
	"r_64kt_nw",
	"r_max_sus"
]
for col in float_cols:
	df[col] = df[col].astype('float')

# Convert null placeholder values to nan
for col in float_cols:
	mask = df[col] < 0
	df.loc[mask, col] = np.nan

# Drop last rows that have no data
df = df[~df['status'].isna()]

# convert to datetime
df['observation_datetime'] = pd.to_datetime(df['date_string']) + pd.to_timedelta(df['time_24h'].astype(int) / 100, unit='h')

# add date of formation to every row
dates_of_formation = df.groupby('code').agg({'observation_datetime':'min'})
dates_of_formation.columns = ['formation_datetime']
df = df.merge(right=dates_of_formation, how='left', on='code')

# convert lat/lon to NOAA format
df['lat'] = df['lat_string'].apply(str2deg_lat)
df['lon'] = df['lon_string'].apply(str2deg_lon)

# drop all storms that were never tropical
# status_mask = df['status'].isin(['HU','TS','TD'])
# code_mask = df['code'].isin(df[status_mask].code.unique())
# df = df[code_mask]

# drop vestigial columns
df.drop(axis=1, labels=["UNUSED", "n_observations", "date_string", "time_24h", "lat_string", "lon_string"], inplace=True)

In [ ]:
df.to_csv("data/hurdat2_cleaned.csv", index=False)

# Import with:
# df = pd.read_csv("data/hurdat2_cleaned.csv", parse_dates=["observation_datetime","formation_datetime"])